In [ ]:
import os
import sys
os.chdir("../../../")
print("현재 작업 디렉토리:", os.getcwd())
import matplotlib.pyplot as plt
from matplotlib import font_manager
font_path = 'font/NanumGothic.ttf'
fontprop = font_manager.FontProperties(fname=font_path)
plt.rc('font', family=fontprop.get_name())
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt  # 시각화 라이브러리 임포트
plt.rc('font', family='NanumGothic')  # 한글 폰트 설정 (나눔고딕, 시스템에 설치 필요)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [ ]:
import warnings  # 경고 메시지 억제용 모듈 임포트
warnings.filterwarnings('ignore')  # 모든 경고 메시지 출력 억제

# 🧪 전체 파이프라인 통합 테스트

이 노트북은 전체 학습/추론 파이프라인의 통합 동작을 테스트합니다:
- 데이터셋, 모델, 학습, 추론 모듈 연동
- 주요 입출력 및 로그 확인

In [ ]:
import os  # OS 모듈 임포트
os.chdir("../../../")  # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())  # 현재 디렉토리 출력

In [ ]:
import torch  # 파이토치 임포트
from src.pipeline.full_pipeline import run_full_pipeline  # 전체 파이프라인 함수 임포트
from src.utils.common import load_yaml  # 설정 로드 함수 임포트
from src.utils.unit_test_logger import create_test_logger  # 단위 테스트 로거 생성 함수 임포트
test_logger = create_test_logger("pipeline_integration")  # 단위 테스트 로거 초기화

In [ ]:
# GPU 자동 체크 코드
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

In [ ]:
# 전체 파이프라인 실행 테스트
try:  # 예외 처리 시작
    cfg = load_yaml("configs/train_highperf.yaml")  # 설정 파일 로드
    result = run_full_pipeline(cfg)  # 전체 파이프라인 실행
    print(f'✅ 파이프라인 실행 결과: {result}')  # 결과 출력
except Exception as e:  # 예외 발생 시
    print(f'⚠️ 파이프라인 실행 실패: {e}')  # 에러 메시지 출력

## 🏆 테스트 결과 요약

- 전체 파이프라인 실행 결과와 주요 입출력, 로그를 확인했습니다.
- 각 단계별 정상 동작 시 ✅, 실패 시 ⚠️ 메시지로 확인 가능합니다.
- 테스트 결과 및 로그는 코드 셀의 출력과 test_logger에 기록됩니다.
- 추가 검증이 필요한 부분은 각 셀의 예외 메시지와 로그를 참고하세요.